In [1]:
import sqlite3

In [2]:
START_YEAR = 2005
CURRENT_SEASON = 2024
MONTHS = [4,5,6,7,8,9] # March folded into April, October into September

In [3]:
db = sqlite3.connect("BaseballStats.db")
cursor = db.cursor()

In [4]:
from tqdm import tqdm

In [5]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
cursor.execute("DELETE FROM Player_Pitcher_MonthlyRatios")

playerData = cursor.execute("SELECT * FROM Player_Pitcher_MonthStats ORDER BY mlbId DESC, Year DESC, Month DESC, Level DESC").fetchall()

for mlbId, year, month, level, pa, outs, go, ao, r, er, h, k, bb, hbp, double, triple, hr, _, _ in tqdm(playerData):
    try:
        _, _, _, levelEra, levelRA, levelFipConstant, levelWOBA, levelHRPerc, levelBBPerc, levelKPerc, levelGOPerc, levelAvg, levelIso = cursor.execute("SELECT * FROM Level_PitcherStats WHERE Level=? AND Year=? AND Month=?", (level, year, month)).fetchone()
    except: # No Data
        continue
    
    singles = h - double - triple - hr
    if pa == 0:
        hrPerc = levelHRPerc
        bbPerc = levelBBPerc
        kPerc = levelKPerc
        wOBA = levelWOBA
    else:
        # https://library.fangraphs.com/offense/woba/
        wOBA = (0.69 * bb + 0.72 * hbp + 0.89 * singles + 1.27 * double + 1.62 * triple + 2.10 * hr) / (pa)
        hrPerc = hr / pa
        bbPerc = bb / pa
        kPerc = k / pa
    
    if outs == 0:
        goPerc = levelGOPerc
        if er == 0:
            era = levelEra
        else:
            era = er * 27
        fip = era
    else:
        if (go + ao) == 0:
            goPerc = levelGOPerc
        else:
            goPerc = go / (go + ao)
        era = (er / outs) * 27
        fip = (13 * hr + 3 * (bb + hbp) - 2 * k) / (outs / 3) + levelFipConstant
    
    wobaRatio = wOBA / levelWOBA
    hrPercRatio = hrPerc / levelHRPerc
    bbPercRatio = bbPerc / levelBBPerc
    kPercRatio = kPerc / levelKPerc
    gbPercRatio = goPerc / levelGOPerc
    eraRatio = era / levelEra
    fipRatio = fip / levelEra
    
    cursor.execute("INSERT INTO Player_Pitcher_MonthlyRatios VALUES(?,?,?,?,?,?,?,?,?,?,?)", (mlbId, year, month, level, gbPercRatio, eraRatio, fipRatio, wobaRatio, hrPercRatio, bbPercRatio, kPercRatio))
    
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 382114/382114 [00:03<00:00, 122147.44it/s]
